QUI CERCO DI OTTENERE IL SESSO E LA DATA DI NASCITA DEL CONDUTTORE DI SANREMO

In [2]:
import pandas as pd
import wikipedia
import re
import concurrent.futures
from tqdm import tqdm

# Imposta la lingua italiana
wikipedia.set_lang("it")

# Carica il file Excel
file_path = "File Unico Sanremo.xlsx"
df = pd.read_excel(file_path, sheet_name="dati-festival-sanremo-1951-2023")

# Pulisce i dati dalla colonna Presentatore
df = df.dropna(subset=["Presentatore"])
df["Presentatore"] = df["Presentatore"].astype(str).str.strip()

# Estrae nomi singoli da stringhe con più conduttori
presentatori_unici = set()
for entry in df["Presentatore"]:
    for nome in re.split(r",\s*|\s+e\s+", entry):
        presentatori_unici.add(nome.strip())

# Crea DataFrame unico
df_presentatori = pd.DataFrame(presentatori_unici, columns=["Nome"])

# Cache
summary_cache = {}
dob_cache = {}

# Genere
def determina_genere(nome):
    if nome in summary_cache:
        return summary_cache[nome]
    try:
        print(f"🟦 Estrazione genere per: {nome}")
        summary = wikipedia.summary(nome, sentences=2).lower()
        summary_cache[nome] = summary
    except Exception as e:
        print(f"❌ Errore Wikipedia (genere) su: {nome} — {e}")
        return "Da verificare"

    if any(x in summary for x in ["una conduttrice", "una presentatrice", "una giornalista", "voce femminile"]):
        return "Femmina"
    elif any(x in summary for x in ["un conduttore", "un presentatore", "un giornalista", "voce maschile"]):
        return "Maschio"
    elif "gruppo" in summary or "duo" in summary:
        return "Gruppo"
    else:
        return "Da verificare"

# Data di nascita
def estrai_data_nascita(nome):
    if nome in dob_cache:
        return dob_cache[nome]
    try:
        print(f"🟨 Estrazione data di nascita per: {nome}")
        content = wikipedia.page(nome).content.lower()
    except Exception as e:
        print(f"❌ Errore Wikipedia (nascita) su: {nome} — {e}")
        return "Da verificare"

    match = re.search(r"(nato|nata) il (\d{1,2}) ([a-z]+) (\d{4})", content)
    if match:
        giorno = match.group(2).zfill(2)
        mese = match.group(3)
        anno = match.group(4)
        data = f"{giorno} {mese} {anno}"
        dob_cache[nome] = data
        return data
    else:
        print(f"⚠️ Nessuna data trovata per: {nome}")
        return "Da verificare"

# Funzione con tracciamento
def apply_parallel(lista, func, descrizione):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(func, nome): nome for nome in lista}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc=descrizione):
            nome = futures[future]
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                print(f"❌ Errore durante {descrizione} su: {nome} — {e}")
                results.append("Errore")
    return results

# Applica le funzioni
df_presentatori["Genere"] = apply_parallel(df_presentatori["Nome"], determina_genere, "Estrazione genere")
df_presentatori["Data di nascita"] = apply_parallel(df_presentatori["Nome"], estrai_data_nascita, "Estrazione date")

# Salva
output_file = "Presentatori_con_genere_e_datanascita.xlsx"
df_presentatori.to_excel(output_file, index=False)

print(f"✅ File generato: {output_file}")


🟦 Estrazione genere per: Enzo Tortora
🟦 Estrazione genere per: Antonella Clerici
🟦 Estrazione genere per: Gianni Morandi
🟦 Estrazione genere per: Giorgio Panariello
🟦 Estrazione genere per: Simona Ventura
🟦 Estrazione genere per: Mike Bongiorno
🟦 Estrazione genere per: Pippo Baudo
🟦 Estrazione genere per: Raffaella Carrà
🟦 Estrazione genere per: Miguel Bosé
🟦 Estrazione genere per: Andrea Occhipinti


Estrazione genere:  21%|██        | 7/34 [00:01<00:05,  5.20it/s]

🟦 Estrazione genere per: Corrado (conduttore televisivo)
🟦 Estrazione genere per: Nuccio Costa
🟦 Estrazione genere per: Roberto Benigni
🟦 Estrazione genere per: Carlo Giuffré
🟦 Estrazione genere per: Gianni Agus
🟦 Estrazione genere per: Lilli Lembo
🟦 Estrazione genere per: Renato Tagliani
🟦 Estrazione genere per: Loretta Goggi
🟦 Estrazione genere per: Claudio Cecchetto
🟦 Estrazione genere per: Rosita Celentano


Estrazione genere:  47%|████▋     | 16/34 [00:03<00:02,  6.28it/s]

🟦 Estrazione genere per: Paolo Bonolis
🟦 Estrazione genere per: Raimondo Vianello
🟦 Estrazione genere per: Fausto Tommei
🟦 Estrazione genere per: Paolo Ferrari (attore)
🟦 Estrazione genere per: Fabio Fazio
🟦 Estrazione genere per: Armando Pizzo
🟦 Estrazione genere per: Amadeus (conduttore televisivo)
🟦 Estrazione genere per: Andrea Giordana
🟦 Estrazione genere per: Carlo Conti (conduttore televisivo)
🟦 Estrazione genere per: Claudio Baglioni


...

⚠️ Nessuna data trovata per: Fausto Tommei
🟨 Estrazione data di nascita per: Nunzio Filogamo
⚠️ Nessuna data trovata per: Paolo Bonolis
🟨 Estrazione data di nascita per: Giancarlo Guardabassi
⚠️ Nessuna data trovata per: Raimondo Vianello
🟨 Estrazione data di nascita per: Gabriella Carlucci
⚠️ Nessuna data trovata per: Armando Pizzo
🟨 Estrazione data di nascita per: Maria Giovanna Elmi
⚠️ Nessuna data trovata per: Andrea Giordana
⚠️ Nessuna data trovata per: Paolo Ferrari (attore)
⚠️ Nessuna data trovata per: Fabio Fazio


Estrazione date:  85%|████████▌ | 29/34 [00:03<00:00, 10.86it/s]

⚠️ Nessuna data trovata per: Carlo Conti (conduttore televisivo)
⚠️ Nessuna data trovata per: Claudio Baglioni


Estrazione date: 100%|██████████| 34/34 [00:04<00:00,  8.23it/s]

⚠️ Nessuna data trovata per: Nunzio Filogamo
⚠️ Nessuna data trovata per: Gabriella Carlucci
⚠️ Nessuna data trovata per: Maria Giovanna Elmi
⚠️ Nessuna data trovata per: Giancarlo Guardabassi
✅ File generato: Presentatori_con_genere_e_datanascita.xlsx
